### Layers and Modules

In [1]:
import tensorflow as tf

In [2]:
class MLP(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
    self.out = tf.keras.layers.Dense(units=10)

  def call(self, X):
    return self.out(self.hidden(X))

In [4]:
X = tf.random.uniform(shape=(2, 20))
net = MLP()
net(X).shape

TensorShape([2, 10])

In [5]:
class MySequential(tf.keras.Model):
  def __init__(self, *args):
    super().__init__()
    self.modules = args

  def call(self, X):
    for module in self.modules:
      X = module(X)
    return X

In [7]:
net = MySequential(
  tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
  tf.keras.layers.Dense(10)
)
net(X).shape

TensorShape([2, 10])

In [10]:
class FixedHiddenMLP(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.flatten = tf.keras.layers.Flatten()
    self.rand_weight = tf.constant(tf.random.uniform(shape=(20, 20)))
    self.dense = tf.keras.layers.Dense(units=20, activation=tf.nn.relu)
  
  def call(self, inputs):
    X = self.flatten(inputs)
    X = tf.nn.relu(tf.matmul(X, self.rand_weight) + 1)
    X = self.dense(X)
    while tf.reduce_sum(tf.math.abs(X)) > 0:
      X /= 2
    return tf.reduce_sum(X)


In [11]:
net = FixedHiddenMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

## Custom Layers

In [12]:
class Dense(tf.keras.Model):
  def __init__(self, units):
    super().__init__()
    self.units = units

  def build(self, X_shape):
    self.weight = self.add_weight(name='weight', shape=[X_shape[-1], self.units], 
                                  initializer=tf.random_normal_initializer())
    self.bias = self.add_weight(name='bias', shape=[self.units], 
                                  initializer =tf.zeros_initializer())
  
  def call(self, X):
    linear = tf.matmul(X, self.weight) + self.bias
    return tf.nn.relu(linear)

In [13]:
dense = Dense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[ 0.03132886, -0.00031938, -0.01998461],
        [ 0.02141164, -0.0212354 , -0.01568849],
        [-0.08186897, -0.01432206, -0.07969992],
        [-0.03044837,  0.02433801,  0.00246944],
        [ 0.00528436, -0.03039438,  0.01373704]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [14]:
net = tf.keras.models.Sequential([Dense(8), Dense(1)])
net(tf.random.uniform(shape=(2, 64)))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.],
       [0.]], dtype=float32)>

In [15]:
class MLP(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.flatten = tf.keras.layers.Flatten()
    self.hidden = Dense(units=256)
    self.out = tf.keras.layers.Dense(units=10)

  def call(self, inputs):
    x = self.flatten(inputs)
    x = self.hidden(x)
    return self.out(x)
  
net = MLP()
X = tf.random.uniform(shape=(2, 20))
Y = net(X)

In [16]:
net.save_weights('mlp.params')

In [17]:
clone = MLP()
clone.load_weights('mlp.params')